# Challenge: Análisis de Tweets

En este desafío, vamos a realizar un análisis de datos sobre un conjunto de tweets relacionados con las protestas de agricultores. Utilizaremos Python y algunas herramientas como `unittest`, `cProfile`, `memory_profiler` y `Jupyter Notebook` para llevar a cabo este análisis.

## 1. Preparación del entorno

Antes de empezar, necesitamos asegurarnos de tener todas las herramientas y datos necesarios. Para esto, verificamos que tenemos los siguientes archivos:

- `q1_memory.py`: Contiene la implementación de la función `q1_memory` que busca encontrar las fechas y usuarios más activos en función del conteo de tweets.
- `q1_time.py`: Contiene la implementación de la función `q1_time` que realiza la misma tarea que `q1_memory` pero midiendo el tiempo de ejecución.
- `q2_memory.py`: Contiene la implementación de la función `q2_memory` que busca encontrar los emojis más utilizados en los tweets.
- `q2_time.py`: Contiene la implementación de la función `q2_time` que realiza la misma tarea que `q2_memory` pero midiendo el tiempo de ejecución.
- `q3_memory.py`: Contiene la implementación de la función `q3_memory` que busca encontrar los usuarios más influyentes en función del conteo de menciones.
- `q3_time.py`: Contiene la implementación de la función `q3_time` que realiza la misma tarea que `q3_memory` pero midiendo el tiempo de ejecución.
- `test_q1_memory.py`, `test_q1_time.py`, `test_q2_memory.py`, `test_q2_time.py`, `test_q3_memory.py`, `test_q3_time.py`: Archivos de pruebas unitarias para las funciones mencionadas anteriormente.
- `farmers-protest-tweets-2021-2-4.json`: Archivo JSON con datos de tweets de las protestas de agricultores.

- Se crea el entorno virtual y se activa (macOS):

In [ ]:
python -m venv .venv
source .venv/bin/activate

- Se instalan los requerimientos en el entorno virtual:   

In [ ]:
pip install -r requirements.txt 

- Ahora, importamos las librerías y funciones necesarias:

In [ ]:
import json
import datetime
import memory_profiler
from collections import Counter
from datetime import datetime
from typing import List, Tuple
from q1_memory import q1_memory
from q1_time import q1_time
from q2_memory import q2_memory
from q2_time import q2_time
from q3_memory import q3_memory
from q3_time import q3_time

Se cargan los datos desde JSON

In [ ]:
# Archivo de datos de prueba
test_file_path = '../data/farmers-protest-tweets-2021-2-4.json'

- Se crea la función q1_memory:

In [ ]:
def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    """
    Encuentra las 10 fechas principales con más tweets y los usuarios más activos en esas fechas.

    Args:
    - file_path (str): Ruta al archivo JSON que contiene los datos de los tweets.

    Returns:
    - List[Tuple[datetime.date, str]]: Una lista de tuplas que contiene la fecha y el usuario más activo para cada una de las 10 fechas principales.
    """
    # Leer el archivo JSON
    with open(file_path, 'r') as f:
        data = [[json.loads(line)['date'].split('T')[0], json.loads(line)['user']['username']] for line in f.readlines()]

    # Validar si hay datos
    if not data:
        return []

    # Obtener las 10 fechas más comunes
    most_common_dates = Counter([d[0] for d in data]).most_common(10)

    # Obtener el usuario más activo por fecha
    most_common_users = []
    for date in most_common_dates:
        users_for_date = [d[1] for d in data if d[0] == date[0]]
        if users_for_date:
            most_common_users.append(Counter(users_for_date).most_common(1)[0][0])
        else:
            # Manejar el caso de una fecha sin usuarios
            most_common_users.append(None)

    # Formatear las fechas
    formatted_dates = [datetime.strptime(date[0], "%Y-%m-%d").date() for date in most_common_dates]

    # Agrupar las fechas y usuarios
    results = list(zip(formatted_dates, most_common_users))

    return results


- Prueba de la función q1_memory
Primero, vamos a probar la función q1_memory que encuentra las fechas y usuarios más activos en función del conteo de tweets. Esta función no mide el tiempo de ejecución, sino que se centra en el uso de memoria.

In [ ]:
# Medir el uso de memoria de q1_memory con memory-profiler
mem_usage = memory_profiler.memory_usage((q1_memory, (test_file_path,)))

# Imprimir el uso de memoria
print(f"Uso de memoria de q1_memory: {max(mem_usage)} MB")

# Ejecutar q1_memory y obtener los resultados
result = q1_memory(test_file_path)

# Imprimir los resultados obtenidos
print("Resultados obtenidos:")
for i, (date, username) in enumerate(result, start=1):
    print(f"{i}. Fecha: {date}, Usuario con más publicaciones: {username}")

# Verificar el tipo y formato de los resultados
assert isinstance(result, list)
for item in result:
    assert isinstance(item, tuple)
    assert isinstance(item[0], datetime.date)
    assert isinstance(item[1], str)